In [ ]:
import os
import numpy as np
import pandas as pd
import sys
import csv
import random
import tensorflow as tf
from keras import optimizers
from keras.callbacks import ModelCheckpoint, Callback
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, dot, multiply, Lambda
from keras.models import Model, Sequential, load_model
from keras.layers.wrappers import TimeDistributed
from keras.utils import to_categorical
from keras import backend as K
from random import shuffle
from sklearn.utils import class_weight
from datetime import datetime
random.seed(999)

In [ ]:
from data_generator import *

## Generate data

In [ ]:
## generate linear training and test data
input_train, output_train, input_test, output_test = gen_lin(train_size = 100)

## generate hierarchical training and test data
#input_train, output_train, input_test, output_test = gen_hier(train_size = 100)

## generate ambiguous training and test data
#input_train, output_train = gen_amb(train_size = 100, train_gen = True)
#input_test, output_test_hier, output_test_lin = gen_amb(train_gen=False)

## generate mixed training and test data
#input_train, output_train = gen_mixed(train_gen = True, ratio_amb = 50, ratio_hier = 50)
#input_test, output_test_hier, output_test_lin = gen_mixed(train_gen=False)

## Run model

In [ ]:
y_int = [y.argmax() for y in output_train]

#adjust class weights as a function of training type
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_int), y_int)

# for linear and ambiguous training
class_weights = {0: 0, 1: class_weights[0], 2: class_weights[1], 3: class_weights[2], 
         4: class_weights[3], 5: class_weights[4], 6: class_weights[5], 7: 0, 8: class_weights[6]}

# for hierarchical training
#class_weights = {0: 0, 1: 0, 2: class_weights[0], 3: class_weights[1], 4: class_weights[2], 
#         5: class_weights[3], 6: class_weights[4], 7: class_weights[5], 8: class_weights[6]}

# for mixed training (if ratio is not 100:0 or 0:100)
#class_weights = {0: 0, 1: class_weights[0], 2: class_weights[1], 3: class_weights[2], 
#         4: class_weights[3], 5: class_weights[4], 6: class_weights[5], 7: class_weights[6], 8: class_weights[7]}

vec_in = Input(shape=(4, 57), dtype='float32', name='vec_in')
lstm = LSTM(100, return_sequences=False, name='lstm1')(vec_in)
target_pos = Dense(9, name='target_pos', activation='softmax')(lstm)
model = Model(inputs=vec_in,outputs=target_pos)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(input_train,output_train, steps_per_epoch=100, epochs=50,verbose=1,class_weight=class_weights)

## Save predictions on test set

In [ ]:
predictions = model.predict(x = input_test)
train_type = 'linear' #'hierarchical', 'ambiguous', 'mixed'
train_size = 100

for runs in range(len(input_test)):

    df_pic = pd.DataFrame([[['..'],['..'],['..'],
                            ['...', '...',],['...', '...',],['...', '...',],['...', '...',],
                            ['...', '...',],['...', '...',],['...', '...',],['...', '...',],
                            ['..'],['..'],['..'],['..'],['..'],['..'],['..'],['..'],['..'],
                            ['..'],['..'],['..'],['..'],['..'],['..'],['..'],['..'],
                            ['..'],['..'],['..'],['..'],['..']]], 
                            columns= ['ordinal', 'color', 'shape', 
                                      '1', '2', '3', '4', '5', '6', '7', '8','o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'o7', 'o8', 'o9',
                                      'p1', 'p2', 'p3', 'p4','p5', 'p6', 'p7', 'p8', 'index', 'max', 'correctness','answer_item', 'train_size'])
    
    #gets the index of the highest value
    i_answer = list(predictions[runs]).index(max(predictions[runs]))+1 # index of max value
    p_answer = max(predictions[runs]) # max value
    
    df_pic['index'][0] = i_answer
    df_pic['max'][0] = p_answer
     
    # load input and split into phrase and picture
    ordnum = input_test[runs][0]
    colnum = input_test[runs][1]
    shapenum = input_test[runs][2]
    
    ordnum = list(ordnum)
    colnum = list(colnum)
    shapenum = list(shapenum)
    
    ## the phrase
    
    # ordinal of target phrase
    if ordnum[:6] == [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]:
        ordinal = 'seventh'
    elif ordnum[:6] == [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]:
        ordinal = 'second'
    elif ordnum[:6] == [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]:
        ordinal = 'third'
    elif ordnum[:6] == [0.0, 0.0, 0.0, 1.0, 0.0, 0.0]:
        ordinal = 'fourth'
    elif ordnum[:6] == [0.0, 0.0, 0.0, 0.0, 1.0, 0.0]:
        ordinal = 'fifth'
    elif ordnum[:6] == [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]:
        ordinal = 'sixth'
    else:
        ordinal = 'ordinal?'

    # check what kind of answer the model produced
    if train_type == 'linear':
        if i_answer == list(output_test[runs]).index(max(output_test[runs]))+1:
            if i_answer == 9:
                synt_ans = 'absence correct'
            else:
                synt_ans = 'linear'
        else:
            synt_ans = 'error'        
    elif train_type == 'hierarchical':
        if i_answer == list(output_test[runs]).index(max(output_test[runs]))+1:
            if i_answer == 9:
                synt_ans = 'absence correct'
            else:
                synt_ans = 'hierarchical'
        else:
            synt_ans = 'error'
    else:
        output_test = output_test_hier
        if i_answer == list(output_test_hier[runs]).index(max(output_test_hier[runs]))+1:
            if i_answer == 9:
                synt_ans = 'absence correct'
            else:
                synt_ans = 'hierarchical'
        elif i_answer == list(output_test_lin[runs]).index(max(output_test_lin[runs]))+1:
            synt_ans = 'linear'
        else:
            synt_ans = 'error'
 
    # color of target phrase
    if colnum[:8] == [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]:
        color = 'blue'
    elif colnum[:8] == [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]:
        color = 'green'
    else:
        color = 'color?'
    
    # shape of target phrase
    if shapenum[:9] == [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]:
        shape = 'ball'
    else:
        shape = 'shape?'
    
    ## the picture

    # load input and split into phrase and picture
    picnum = input_test[runs][3]
    picnum = picnum[9:]

    smallrun = 0
    for elements in range(0,48,6):
        smallrun += 1

        colrange1 = elements
        colrange2 = elements+3
        
        shaperange1 = elements+3
        shaperange2 = elements+6

        colnum = list(picnum[colrange1:colrange2])
        if colnum == [0.25, 0.0, 0.0]:
            color_pic = 'blue'
        elif colnum == [0.0, 0.25, 0.0]:
            color_pic = 'green'
        else:
            color_pic = 'color?'

        shapenum = list(picnum[shaperange1:shaperange2])
        if shapenum == [0.0, 0.0, 0.25]:
            shape_pic = 'ball'
        else:
            shape_pic = 'shape?'    

        props = [color_pic, shape_pic]

        df_pic[str(smallrun)][0] = props
    
    df_pic['ordinal'][0] = ordinal
    df_pic['color'][0] = color
    df_pic['shape'][0] = shape
    df_pic['o1'][0] = output_test[runs][0]    
    df_pic['o2'][0] = output_test[runs][1] 
    df_pic['o3'][0] = output_test[runs][2] 
    df_pic['o4'][0] = output_test[runs][3] 
    df_pic['o5'][0] = output_test[runs][4] 
    df_pic['o6'][0] = output_test[runs][5] 
    df_pic['o7'][0] = output_test[runs][6] 
    df_pic['o8'][0] = output_test[runs][7] 
    df_pic['o9'][0] = output_test[runs][8] 
    df_pic['p1'][0] = predictions[runs][0]    
    df_pic['p2'][0] = predictions[runs][1] 
    df_pic['p3'][0] = predictions[runs][2] 
    df_pic['p4'][0] = predictions[runs][3] 
    df_pic['p5'][0] = predictions[runs][4] 
    df_pic['p6'][0] = predictions[runs][5] 
    df_pic['p7'][0] = predictions[runs][6] 
    df_pic['p8'][0] = predictions[runs][7]
    df_pic['correctness'][0] = synt_ans
    if i_answer < 9:
        df_pic['answer_item'][0] = df_pic[str(i_answer)][0] 
    else:
        df_pic['answer_item'][0] = 'target absent'
    df_pic['train_size'][0] = train_size 
   
    if runs == 0:
        phrase_and_pic = pd.DataFrame(df_pic, columns= ['ordinal', 'color', 'shape', '1', '2', '3', '4', '5', '6', '7', '8',
                                                          'o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'o7', 'o8', 'o9',
                                                          'p1', 'p2', 'p3', 'p4','p5', 'p6', 'p7', 'p8', 'index', 'max', 'correctness', 'answer_item', 'train_size'])
    else:
        phrase_and_pic = phrase_and_pic.append(df_pic, ignore_index=True)
    
    
print(phrase_and_pic)

#phrase_and_pic.to_csv('data/' + train_type + '/l_pred_' + str(train_size) + '.csv', index=False)

## Save training data in interpretable format

In [ ]:
train_type = 'linear' #'hierarchical', 'ambiguous', 'mixed'
train_size = 100

for runs in range(len(input_train)):
    
    df_pic = pd.DataFrame([[['..'],['..'],['..'],
                            ['...', '...',],['...', '...',],['...', '...',],['...', '...',],
                            ['...', '...',],['...', '...',],['...', '...',],['...', '...',],
                            ['..'],['..'],['..'],['..'],['..'],['..'],['..'],['..'],['..']]], 
                                                                      columns = ['ordinal', 'color', 'shape', 
                                                                    '1', '2', '3', '4', '5', '6', '7', '8',
                                                                    'o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'o7', 'o8', 'o9'])
    
   
    # load input and split into phrase and picture
    ordnum = input_train[runs][0]
    colnum = input_train[runs][1]
    shapenum = input_train[runs][2]
    
    ordnum = list(ordnum)
    colnum = list(colnum)
    shapenum = list(shapenum)
    
    ## the phrase
    
    # ordinal of target phrase
    if ordnum[:6] == [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]:
        ordinal = 'seventh'
    elif ordnum[:6] == [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]:
        ordinal = 'second'
    elif ordnum[:6] == [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]:
        ordinal = 'third'
    elif ordnum[:6] == [0.0, 0.0, 0.0, 1.0, 0.0, 0.0]:
        ordinal = 'fourth'
    elif ordnum[:6] == [0.0, 0.0, 0.0, 0.0, 1.0, 0.0]:
        ordinal = 'fifth'
    elif ordnum[:6] == [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]:
        ordinal = 'sixth'
    else:
        ordinal = 'ordinal?'
           
    # color of target phrase
    if colnum[:8] == [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]:
        color = 'blue'
    elif colnum[:8] == [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]:
        color = 'green'
    else:
        color = 'color?'
    
    # shape of target phrase
    if shapenum[:9] == [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]:
        shape = 'ball'
    else:
        shape = 'shape?'
    
    ## the picture

    # load input and split into phrase and picture
    picnum = input_train[runs][3]
    picnum = picnum[9:]
    
    smallrun = 0
    for elements in range(0,48,6):
        smallrun += 1
        #print(elements)
        colrange1 = elements
        colrange2 = elements+3
        
        shaperange1 = elements+3
        shaperange2 = elements+6

        colnum = list(picnum[colrange1:colrange2])
        if colnum == [0.25, 0.0, 0.0]:
            color_pic = 'blue'
        elif colnum == [0.0, 0.25, 0.0]:
            color_pic = 'green'
        else:
            color_pic = 'color?'

        shapenum = list(picnum[shaperange1:shaperange2])
        if shapenum == [0.0, 0.0, 0.25]:
            shape_pic = 'ball'
        else:
            shape_pic = 'shape?'    

        props = [color_pic, shape_pic]

        df_pic[str(smallrun)][0] = props
    
    df_pic['ordinal'][0] = ordinal
    df_pic['color'][0] = color
    df_pic['shape'][0] = shape
    df_pic['o1'][0] = output_train[runs][0]    
    df_pic['o2'][0] = output_train[runs][1] 
    df_pic['o3'][0] = output_train[runs][2] 
    df_pic['o4'][0] = output_train[runs][3] 
    df_pic['o5'][0] = output_train[runs][4] 
    df_pic['o6'][0] = output_train[runs][5] 
    df_pic['o7'][0] = output_train[runs][6] 
    df_pic['o8'][0] = output_train[runs][7]
    df_pic['o9'][0] = output_train[runs][8] 

    #print(df_pic)
    if runs == 0:
        phrase_and_pic = pd.DataFrame(df_pic, columns = ['ordinal', 'color', 'shape', '1', '2', '3', '4', '5', '6', '7', '8',
                                                          'o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'o7', 'o8', 'o9'])
    else:
        phrase_and_pic = phrase_and_pic.append(df_pic, ignore_index=True)
    
    
print(phrase_and_pic)

#phrase_and_pic.to_csv('data/' + train_type + '/l_training' + str(train_size) + '.csv', index=False)